# Imports

In [1]:
from src.utilities import predict_and_save, split_data, get_method_name, score_method, encode_filename
from src.preprocess import process_missing_values, main_preprocess, create_entity
from sklearn.model_selection import train_test_split
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis, RandomSurvivalForest
import lightgbm as lgb
import pandas as pd
from datetime import datetime

import warnings
import logging

# Régler le logger de Featuretools au niveau ERROR
logging.getLogger('featuretools.entityset').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore",message=".*Ill-conditioned matrix.*")


2025-03-23 14:07:10,307 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "DiversityScore" from "premium_primitives.diversity_score" because a primitive with that name already exists in "nlp_primitives.diversity_score"
2025-03-23 14:07:10,307 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "LSA" from "premium_primitives.lsa" because a primitive with that name already exists in "nlp_primitives.lsa"
2025-03-23 14:07:10,308 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "MeanCharactersPerSentence" from "premium_primitives.mean_characters_per_sentence" because a primitive with that name already exists in "nlp_primitives.mean_characters_per_sentence"
2025-03-23 14:07:10,308 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "NumberOfSentences" from "premium_primi

# Settings

In [2]:
GLOBAL = {
    "cox": {"run": True, "save":False, "shap": False},
    "xgb": {"run": True, "save": False, "shap": False},
    "lgbm": {"run": True, "save":False, "shap": False},
    "rsf": {"run": True, "save":False, "shap": False}
}

PARAMS = {
    "size": 0.7,
    "impute": {"strategy": "median", "sex": False},
    #"outliers": {"threshold": 0.01, "multiplier": 1.5},
    "clinical": ["CYTOGENETICS"],#["CYTOGENETICS"], # Possible: ["CYTOGENETICS", "HB/PLT", "logMONOCYTES", "logWBC", "logANC"] ["BM_BLAST+WBC", "BM_BLAST/HB", "HB*PLT", "HB/num_trisomies"]
    "molecular": ["GENE"],#["END-START"], # Possible: ["GENE", "EFFECT", "ALT", "REF", "END-START"]
    "merge": ["featuretools", "gpt"], # Possible: ["featuretools", "gpt"]
    "additional": [
        #['cadd', 'phred'],
        # ['cadd', 'rawscore'],
        # # ['cadd', 'consequence'],
        # # ['cadd', 'bstatistic'],
        # # ['cadd', 'gerp', 'n'],
        # ['cadd', 'phast_cons', 'mammalian'],
        # ['cadd', 'phylop', 'mammalian'],
        # ['snpeff', 'putative_impact'],
        # # ['snpeff', 'rank'],
        # # ['snpeff', 'total'],
         #['cadd', 'exon'],
        # # ['cadd', 'cds', 'rel_cds_pos']
        ],
    "xgb": {
        'loss': 'coxph',
        'max_depth': 2,
        'learning_rate': 0.05,
        'n_estimators': 335,
        'subsample': 0.55,
        'max_features': "sqrt",
        'min_samples_split': 3,
        'min_samples_leaf': 1,
        'min_weight_fraction_leaf': 0,
        'min_impurity_decrease': 0,
        'dropout_rate': 0,
        'warm_start': False,
        'ccp_alpha': 0,
        'random_state': 126
    },
    "lgbm": {
        'max_depth': 2,
        'learning_rate': 0.05,
        'verbose': 0
    },
    "rsf": {
    'n_estimators':200,  # Nombre d'arbres dans la forêt
    'max_depth':None,
    'min_samples_split':50,  # Nombre minimum d'échantillons requis pour splitter un nœud
    'min_samples_leaf':20,  # Nombre minimum d'échantillons par feuille
    'max_features':'sqrt',  # Sélection aléatoire des features
    'n_jobs':-1,  # Utilisation de tous les cœurs disponibles
    }
}

##############################################
# Define the methods used for training
##############################################

size_method = get_method_name("size", PARAMS)
clinical_method = get_method_name("clinical", PARAMS)
molecular_method = get_method_name("molecular", PARAMS)
merge_method = get_method_name("merge", PARAMS)

# Preprocess, Handling missing values, Train/Test split

In [3]:
data = create_entity(PARAMS)
data = main_preprocess(data, PARAMS)
X, X_eval, y = split_data(data)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(1 - PARAMS['size']), random_state=42)
X_train, X_test, X_eval = process_missing_values(X_train, X_test, X_eval, X.columns, **PARAMS["impute"])

# EDA

In [4]:
from src.report import EDAReport
from src.utilities import prepare_EDA

df_analyze = prepare_EDA(X.columns, X_train, y_train)

ede = EDAReport(df_analyze, target_variables=["event", "time"])
ede.generate_report(max_rows=100, max_cols=10)
ede.display()

/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imported v0.1.905. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)
Analyse pour la variable cible 'event'
    Since nrows is smaller than dataset, loading random sample of 100 rows into pandas...
Shape of your Data Set loaded: (100, 104)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    103 Predictors classified...
        3 variable(s) removed since they were ID or low-information variables
Since Number of Rows in data 100 exceeds maximum, randomly sampling 100 rows for EDA...

################ Binary_

Saving scatterplots in HTML format
  0%|          | 0/10 [00:00<?, ?it/s]

 40%|████      | 4/10 [00:00<00:00, 34.83it/s]

 80%|████████  | 8/10 [00:00<00:00, 31.04it/s]

...

Saving kde_plots in HTML format


Saving violinplots in HTML format


Saving heatmaps in HTML format


Saving cat_var_plots in HTML format
Time to run AutoViz (in seconds) = 3          
Le rapport EDA a été sauvegardé dans : report/eda/EDA_20250323_140716/EDA_report.html


127.0.0.1 - - [23/Mar/2025 14:07:24] "GET /report/eda/EDA_20250323_140716/EDA_report.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2025 14:07:24] "GET /report/eda/EDA_20250323_140716/event/distplots_nums.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2025 14:07:24] "GET /report/eda/EDA_20250323_140716/event/heatmaps.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2025 14:07:24] "GET /report/eda/EDA_20250323_140716/event/pair_scatters.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2025 14:07:24] "GET /report/eda/EDA_20250323_140716/event/scatterplots.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2025 14:07:24] "GET /report/eda/EDA_20250323_140716/event/violinplots.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2025 14:07:24] "GET /report/eda/EDA_20250323_140716/time/cat_var_plots.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2025 14:07:24] "GET /report/eda/EDA_20250323_140716/time/distplots_cats.html HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2025 14:07:24] "GET /report/eda/EDA_20250323_140716/time/distplots_nums.htm

# Models

## Fit a CoxPH model

In [6]:
if GLOBAL["cox"]["run"]:

    cox = CoxPHSurvivalAnalysis()
    cox.fit(X_train, y_train)
    cox_score_method = score_method(cox, X_train, X_test, y_train, y_test)

    if GLOBAL["cox"]["save"] or GLOBAL["cox"]["shap"]:
        method = f"{size_method}-{cox_score_method}-{clinical_method}-{molecular_method}-{merge_method}"
        name = f'{cox.__class__.__name__}_submission_{method}_{datetime.now().strftime("%Y%m%d%H%M%S")}'
        token = encode_filename(name, token_length=40)

    if GLOBAL["cox"]["save"]:
        predict_and_save(X_eval, cox, name=token)

    if GLOBAL["cox"]["shap"]:
        from src.report import ShapReport

        if not isinstance(X_train, pd.DataFrame):
            X_train = pd.DataFrame(X_train, columns=X.columns)

        report_cox = ShapReport(model=cox, X_train=X_train, predict_function=cox.predict)
        report_cox.generate_report(output_file=f"report/shap/shap_{token}.html")
        report_cox.display()

CoxPHSurvivalAnalysis Model Concordance Index IPCW on train: 0.730
CoxPHSurvivalAnalysis Model Concordance Index IPCW on test: 0.687


## Fit a LightGBM model

In [ ]:
if GLOBAL["lgbm"]["run"]:
    lgbm_params_method = "_".join([f"{key}={PARAMS['lgbm'][key]}" for key in PARAMS['lgbm'].keys()])

    train_dataset = lgb.Dataset(X_train, label=y_train['time'])
    model_lgb = lgb.train(params=PARAMS['lgbm'], train_set=train_dataset)
    lightgbm_score_method = score_method(model_lgb, X_train, X_test, y_train, y_test, reverse=True)

    if GLOBAL["lgbm"]["save"] or GLOBAL["lgbm"]["shap"]:
        method = f"{size_method}-{lightgbm_score_method}-{clinical_method}-{molecular_method}-{merge_method}-{lgbm_params_method}"
        name = f'{model_lgb.__class__.__name__}_submission_{method}_{datetime.now().strftime("%Y%m%d%H%M%S")}'
        token = encode_filename(name, token_length=40)

    if GLOBAL["lgbm"]["save"]:
        predict_and_save(X_eval, model_lgb, name=token)

    if GLOBAL["lgbm"]["shap"]:
        from src.report import ShapReport

        if not isinstance(X_train, pd.DataFrame):
            X_train = pd.DataFrame(X_train, columns=X.columns)

        report_lgbm = ShapReport(model=model_lgb, X_train=X_train, predict_function=model_lgb.predict)
        report_lgbm.generate_report(output_file=f"report/shap/shap_{token}.html")
        report_lgbm.display()

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Booster Model Concordance Index IPCW on train: 0.273
Booster Model Concordance Index IPCW on test: 0.311


## Fit a Gradient Boosting Survival Analysis Model

In [8]:
if GLOBAL["xgb"]["run"]:
    xgb_params_method = "_".join([(str(key) + "=" + str(PARAMS['xgb'][key])) for key in PARAMS['xgb'].keys()])

    xgb = GradientBoostingSurvivalAnalysis(**PARAMS['xgb'])
    xgb.fit(X_train, y_train)
    xgboost_score_method = score_method(xgb, X_train, X_test, y_train, y_test)

    if GLOBAL["xgb"]["save"] or GLOBAL["xgb"]["shap"]:
        method = f"{size_method}-{xgboost_score_method}--{molecular_method}-{merge_method}-{xgb_params_method}"
        name = f'{xgb.__class__.__name__}_submission_{method}_{datetime.now().strftime("%Y%m%d%H%M%S")}'

        token = encode_filename(name, token_length=40)

    if GLOBAL["xgb"]["save"]:
        predict_and_save(X_eval, xgb, name=token)

    if GLOBAL["xgb"]["shap"]:
        from src.report import ShapReport

        if not isinstance(X_train, pd.DataFrame):
            X_train = pd.DataFrame(X_train, columns=X.columns)

        report_xgb = ShapReport(model=xgb, X_train=X_train, predict_function=xgb.predict)
        report_xgb.generate_report(output_file=f"report/shap/shap_{token}.html")
        report_xgb.display()

GradientBoostingSurvivalAnalysis Model Concordance Index IPCW on train: 0.752
GradientBoostingSurvivalAnalysis Model Concordance Index IPCW on test: 0.701


## Fit a Random Survival Forest model

In [9]:
if GLOBAL["rsf"]["run"]:
    rsf_params_method = "_".join([(str(key) + "=" + str(PARAMS['rsf'][key])) for key in PARAMS['rsf'].keys()])

    rsf = RandomSurvivalForest(**PARAMS["rsf"], random_state=42)
    rsf.fit(X_train, y_train)
    rsf_score_method = score_method(rsf, X_train, X_test, y_train, y_test)

    if GLOBAL["rsf"]["save"] or GLOBAL["rsf"]["shap"]:
        method = f"{size_method}-{rsf_score_method}-{clinical_method}-{molecular_method}-{merge_method}-{rsf_params_method}"
        name = f'{rsf.__class__.__name__}_submission_{method}_{datetime.now().strftime("%Y%m%d%H%M%S")}'
        token = encode_filename(name, token_length=40)

    if GLOBAL["rsf"]["save"]:
        predict_and_save(X_eval, rsf, name=token)

    if GLOBAL["rsf"]["shap"]:
        from src.report import ShapReport

        if not isinstance(X_train, pd.DataFrame):
            X_train = pd.DataFrame(X_train, columns=X.columns)

        report_rsf = ShapReport(model=rsf, X_train=X_train, predict_function=rsf.predict)
        report_rsf.generate_report(output_file=f"report/shap/shap_{token}.html")
        report_rsf.display()

RandomSurvivalForest Model Concordance Index IPCW on train: 0.766
RandomSurvivalForest Model Concordance Index IPCW on test: 0.695


# Deep

## Custom second preprocess

In [10]:
from src.deep import convert_float32, convert_survival_data, score_method_deep

X_train_deep, X_test_deep, X_eval_deep = convert_float32(X_train, X_test, X_eval)

y_train_deep = convert_survival_data(y_train)
y_test_deep = convert_survival_data(y_test)

## Parameters

In [11]:
params = {
    'num_nodes': [128, 128, 23],  # Augmentation de la capacité du modèle
    'out_features': 1,
    'batch_norm': True,
    'dropout': 0.0,              # Vous pouvez tester avec ou sans dropout
    'output_bias': False,
    'in_features': X_train_deep.shape[1]
}
params_CoxHP = {
    "batch_size": 256,
    "lr": 0.0005,              # Taux d'apprentissage légèrement réduit
    "epochs": 512,             # Augmentation du nombre d'époques
    "verbose": False
}

## Fit a CoxHP

In [ ]:
from sklearn.preprocessing import StandardScaler
import torch
import torchtuples as tt
from pycox.models import CoxPH
import numpy as np

np.random.seed(42)
_ = torch.manual_seed(4)

params['in_features'] = X_train_deep.shape[1]

net = tt.practical.MLPVanilla(**params)
model = CoxPH(net, tt.optim.Adam)

model.optimizer.set_lr(params_CoxHP['lr'])

batch_size = params_CoxHP['batch_size']

lrfinder = model.lr_finder(X_train_deep, y_train_deep, batch_size, tolerance=10)
_ = lrfinder.plot()

epochs = params_CoxHP['epochs']
callbacks = [tt.callbacks.EarlyStopping()]
verbose = params_CoxHP['verbose']

log = model.fit(X_train_deep, y_train_deep, batch_size, epochs, callbacks, verbose,
                val_data=(X_test_deep, y_test_deep), val_batch_size=batch_size)

_ = log.plot()

print("LogLikehood: ", model.partial_log_likelihood(*(X_test_deep, y_test_deep)).mean())

score_method_deep(model, X_train_deep, X_test_deep, y_train, y_test, reverse=False)

## Make a prediction

In [ ]:
from src.deep import predict_and_save_deep

predict_and_save_deep(X_eval_deep, model, method="deepCoxHP")